## IMP
password for postgres - postgres 

password for mysql - password123

## Database and SQL- Structured Query language

1. __DBMS__: Database management system is a special software program that helps create and maintain databases.
2. __C.R.U.D__ → Create Read/Retrieve Update Delete
3. __Types of databases__:
    1. __Relational database(SQL)__:Organise data in one or more tables related to each other. Each table has columns and rows and a Unique key identifies each row.
    2. __Non-Relational database(noSQL)__:Any database which is not relational is said as non-relational database. It is used in very generic term. Stores data in the form of key value pair. Ex Documents - JSON, XML etc.
4. __RDBMS__: Relational Database Management System helps create and maintain a relational database. Ex: mySQL, PostgreSQL, Oracle, mariaDB. SQL is used as a standard language with RDBM to perform C.R.U.D operations
5. __NRDBMS__: Non-Relational Database Management System helps create and maintain a non-relational database. Ex: mongoDB, dynamoDB, apache cassandra, firebase etc. Most NRDBMS implement their own query language to perform C.R.U.D

# RDBMS
https://www.sqltutorial.org/sql-cheat-sheet/ → Best with examples

1. A ‘table’ is a set of data elements. Each row is called a ‘record’ and each column is called as ‘field’ or ‘attribute’. And each cell has ‘Data’ in it.
2. A Unique value is a value that cannot be repeated.Null is the absence of value 

SQL commands can be categorized into 4 main types:
<img src="images/image66.png" align="middle" style="width:600px; height:400px" />

1. To work with Postgresql Database you can either work on PGAdmin(which is the GUI provided by postgresql) or you can work on SQL shell.
2. By default in shell script the user is the superuser (postgres). Do the following to get it started to work in shell.
<img src="images/image67.png" align="middle" style="width:400px; height:300px" />
3. First statement is to create a schema. Second “SET ..” is the statement is required so that when we command actions (like create table) is to be performed on information schema.

1. Database Schema: The term "schema" refers to the organization of data as a blueprint of how the database is constructed (divided into database tables in the case of relational databases)  
__CREATE SCHEMA information;__   

2.  In a relational database, the schema defines the tables, fields, relationships, views, indexes, packages, procedures, functions, queues, triggers, types, sequences, materialized views, synonyms, database links, directories, XML schemas, and other elements.
3. CREATE TABLE students(studentid int, studentname varchar(255)); → this will create a table named students with studentid and studentname as columns in information schema(as we performed SET to information)

__Normaliztion in Databases__:
1. https://www.essentialsql.com/get-ready-to-learn-sql-database-normalization-explained-in-simple-english/
2. There are three main reasons to normalize a database.  The first is to minimize duplicate data, the second is to minimize or avoid data modification issues, and the third is to simplify queries. 

__Keys__:A DBMS key is an attribute or set of an attribute which helps you to identify a row in a relational table. They allow you to find the relation between two tables. Keys help you uniquely identify a row in a table by a combination of one or more columns in that table.Below are list of types of keys:
1. https://www.studytonight.com/dbms/database-key.php
2. __Super Key__: Super Key is defined as a set of attributes within a table that can uniquely identify each record within a table. Super Key is a superset of Candidate key.
3. __Candidate Key__: Candidate keys are defined as the minimal set of fields which can uniquely identify each record in a table. It is an attribute or a set of attributes that can act as a Primary Key for a table to uniquely identify each record in that table. There can be more than one candidate key.
    1. A candidate key can never be NULL or empty. And its value should be unique.
    2. There can be more than one candidate keys for a table.
    3. A candidate key can be a combination of more than one columns(attributes).
4. __Primary Key__: Primary key is a candidate key that is most appropriate to become the main key for any table. It is a key that can uniquely identify each record in a table.The primary key cannot be null (blank).The primary key is indexed 
5. __Foreign Key__: A foreign key is a column which is added to create a relationship with another table.Foreign keys help us to maintain data integrity and also allows navigation between two different instances of an entity. Every relationship in the model needs to be supported by a foreign key. A foreign key is a value which matches back to another table’s primary key.
6. __Composite Key__: A key which has multiple attributes to uniquely identify rows in a table is called a composite key.
7. __Natural Key__: A composite primary key which is composed of attributes (fields) which already exist in the real world (for example StudentID, Social Security Number).
8. __Surrogate Key__: A composite primary key which is composed of attributes (fields) which already exist in the real world (for example First Name, Last Name, Social Security Number).


__Types of Database relationship__: The power of a relational database is in the links and relations. By connecting rows in different tables through the use of primary and foreign keys, you can create views, reports, and other slices of information to serve your organization. There are three primary types of database relationships:
1. __one-to-one (1:1)__ : One row in table A relates to one row in table B.  Using an entity-relationship (ER) model, 1:1 means that one occurrence of an entity relates to only one occurrence in another entity. Foreign Key(FK) can logical be in any one of the two tables, wherever it makes more sense.
2. __one-to-Many(1:M)__ : One row in table A relates to many rows in table B.  In ER modelling, 1:Many means that one occurrence in an entity relates to many occurrences in another entity.FK should be in many side(Table B), so that table B FK field can exactly relate to one value in Table in A.
3. __many-to-many (M:M)__ : Many rows in table A relate to many rows in table B. In ER terms, many occurrences in one entity relate to many occurrences in another entity. For example, a student (table A) may sign up for many classes (table B), and a class may have several students in it.  Many-to-many relationships normally require a cross-reference table, AB with two one-to-many relationships A:AB and B:AB. In this case FK cannot be in any table, so there comes a third table called cross-reference table which shows the relationship between the two tables and has two FKs.


__Like__: When you don’t know exactly what to search for you can use pattern matching in conditions, by using wildcard characters in a LIKE statement.
1. The percent wildcard (%): any string of zero or more characters.
2. The underscore (_) wildcard: any single character.
3. The [list of characters] wildcard: any single character within the specified set.
4. The [character-character]: any single character within the specified range.
5. The [^]: any single character not within a list or a range.
>SELECT *
>FROM product
>WHERE product_name LIKE "%n"; /*Will search for data ending with the letter ‘n’.*/


__Aggregate Functions__: https://www.sqltutorial.org/sql-aggregate-functions/


__GROUP BY__: https://www.sqltutorial.org/sql-group-by

__HAVING VS WHERE__: The WHERE clause applies the condition to individual rows before the rows are summarized into groups by the GROUP BY clause. However, the HAVING clause applies the condition to the groups after the rows are grouped into groups.


# Woking with MySQL:

https://www.youtube.com/watch?v=7S_tz1z_5bA  
pip install mysql-connector-python  
Make sure mysql server is up and running.

Common commands and tips and tricks for MySql:

1. https://www.sqltutorial.org/sql-cheat-sheet/
2. Open MySQL Workbench to establish a connection.
1. USE database_name; - marks up the database in use.
2. -- ANY COMMAND; - is used as a comment line
3. Commands needs to be written in order:
    1. __SELECT__ column_names
    2. __FROM__ table_name
    3. __WHERE__ condition
    4. __ORDER BY__ columns
6. https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html
7. http://g2pc1.bu.edu/~qzpeng/manual/MySQL%20Commands.htm

In [2]:
import mysql.connector
import os
import pandas as pd

In [172]:
# does not work but this the ideal way so password is not exposed.
# user_name = os.environ.get('USER')
# password = os.environ.get('DB_PASS')
# print(user_name)

In [43]:
connection = mysql.connector.connect(host='localhost',
                                    user = 'root',
                                    passwd = 'password123',
                                    )
connection.connection_id
# auth_plugin='mysql_native_password' # in case if there is authentication plugin 'caching_sha2_password' error

62

In [44]:
# you can chnage the databse anytime to something else by just assigning it like this.
connection.database = 'sql_store'

In [45]:
pd.read_sql_query(
'SHOW databases' \
, connection)

,Database
0,information_schema
1,mysql
2,performance_schema
3,sakila
4,sql_hr
5,sql_inventory
6,sql_invoicing
7,sql_store
8,sys
9,world


In [238]:
pd.read_sql_query(
'SHOW TABLES FROM sql_store'
, connection)
# this also will work
pd.read_sql_query(
'SHOW TABLES FROM sql_inventory'
, connection)

,Tables_in_sql_inventory
0,products


In [178]:
# we can do arthemetic operations on columns like 'points * 10 + 10'
pd.read_sql_query(
'SELECT last_name, first_name, points, points * 10 + 10 \
FROM customers'
, connection)

,last_name,first_name,points,points * 10 + 10
0,MacCaffrey,Babara,2273,22740
1,Brushfield,Ines,947,9480
2,Boagey,Freddi,2967,29680
3,Roseburgh,Ambur,457,4580
4,Betchley,Clemmie,3675,36760
5,Twiddell,Elka,3073,30740
6,Dowson,Ilene,1672,16730
7,Naseby,Thacher,205,2060
8,Rumgay,Romola,1486,14870
9,Mynett,Levy,796,7970


In [179]:
# same of above with alis name
# we can do arthemetic operations on columns like 'points * 10 + 10'
pd.read_sql_query(
'SELECT \
    last_name, \
    first_name, \
    points, \
    (points * 10) + 100 AS disc_fct \
FROM customers'
, connection)
# (points * 10) + 100 AS disc_fct # use this for spaced column names

,last_name,first_name,points,disc_fct
0,MacCaffrey,Babara,2273,22830
1,Brushfield,Ines,947,9570
2,Boagey,Freddi,2967,29770
3,Roseburgh,Ambur,457,4670
4,Betchley,Clemmie,3675,36850
5,Twiddell,Elka,3073,30830
6,Dowson,Ilene,1672,16820
7,Naseby,Thacher,205,2150
8,Rumgay,Romola,1486,14960
9,Mynett,Levy,796,8060


In [180]:
# retrive Unique(DISTINCT) records from customer table - state column.
pd.read_sql_query(
'SELECT DISTINCT state \
FROM customers'
, connection)

,state
0,VA
1,CO
2,FL
3,TX
4,IL
5,TN
6,CA
7,GA


## WHERE Clause

In [181]:
# We can use WHERE with conditional operator like < , >, ==, !==, >=, <= 
# we can combine AND OR NOT with them 
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE points > 3000 AND birth_date > "1960-01-01" '\
, connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [182]:
# we can use IN and NOT IN
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE state IN ("IL", "VA")' \
, connection)
# WHERE state NOT IN ("IL", "VA")' \

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,VA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [183]:
# BETWEEN
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE points BETWEEN 1000 AND 2000 ' \
, connection)
# WHERE state BETWEEN "A" AND "L" ' \ # Not a suitable example but just for the sake

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [184]:
# Use LIKE or NOT LIKE to match a specific string pattern
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE last_name LIKE "%y" ' \
, connection)
# ------------------------
# %  any character
# _ any one character
# ------------------------
# pd.read_sql_query(
# 'SELECT * \
# FROM customers \
# WHERE last_name LIKE "_____y" ' \
# , connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,VA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [185]:
# more powerful than LIKE is REGEXP
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE last_name REGEXP "field$|^mac|rose" ' \
, connection)
# ^ beginning of the string
# $ end of the string
# [aeiou]$ match any single character listed in the []
# -------------------------------------
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE last_name REGEXP "[aeiou]a" ' \
, connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [186]:
# identifying the NULL values use IS and NOT IS
pd.read_sql_query(
'SELECT * \
FROM customers \
WHERE phone IS NULL ' \
, connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


## ORDER BY and OFFSET/LIMIT Clause

In [187]:
# Irrespective of what column is selected in SELECT we can order by the table with any column
pd.read_sql_query(
'SELECT * \
FROM customers \
ORDER BY first_name' \
, connection)
# ORDER BY first_name DESC' \ # for descending order
# ORDER BY state DESC, first_name ' \ # order first state in DESC then frist_name
# ORDER BY can have arthematic expression
# pd.read_sql_query(
# 'SELECT *, points * customer_id AS new \
# FROM customers \
# ORDER BY points * customer_id' \
# , connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,VA,2273
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [188]:
# LIMIT offset, limit # this skips offset number of records and then picks 'limit' after the offset.
# LIMIT should come at last
pd.read_sql_query(
'SELECT * \
FROM customers \
ORDER BY first_name \
LIMIT 3, 2' \
, connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


# JOINS

In [189]:
pd.read_sql_query(
'SELECT * \
FROM orders \
LIMIT 10' \
, connection)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [190]:
pd.read_sql_query(
'SELECT * \
FROM customers \
LIMIT 6' \
, connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,VA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [191]:
# Basic JOIN == INNER JOIN
# when there are ambigious column names in two tables prefix with table name.
pd.read_sql_query(
'SELECT  order_id, first_name, orders.customer_id, customers.customer_id \
FROM orders \
JOIN customers ON orders.customer_id = customers.customer_id \
ORDER BY order_id' \
, connection)
# using alisa to orders table and customer table
# pd.read_sql_query(
# 'SELECT  order_id, first_name, o.customer_id \
# FROM orders o \
# JOIN customers c ON o.customer_id = c.customer_id \
# ORDER BY order_id' \
# , connection)

,order_id,first_name,customer_id,customer_id
0,1,Elka,6,6
1,2,Ilene,7,7
2,3,Thacher,8,8
3,4,Ines,2,2
4,5,Clemmie,5,5
5,6,Levy,10,10
6,7,Ines,2,2
7,8,Clemmie,5,5
8,9,Levy,10,10
9,10,Elka,6,6


# Joining across different Database


In [192]:
pd.read_sql_query(
'SHOW TABLES FROM sql_store'
, connection)

,Tables_in_sql_store
0,customers
1,order_item_notes
2,order_items
3,order_statuses
4,orders
5,products
6,shippers


In [193]:
pd.read_sql_query(
'SHOW databases'
, connection)

,Database
0,information_schema
1,mysql
2,performance_schema
3,sakila
4,sql_hr
5,sql_inventory
6,sql_invoicing
7,sql_store
8,sys
9,world


In [194]:
pd.read_sql_query(
'SHOW TABLES FROM sql_inventory'
, connection)

,Tables_in_sql_inventory
0,products


In [235]:
pd.read_sql_query(
'SELECT  * \
FROM order_items io \
JOIN sql_inventory.products p ON io.product_id = p.product_id \
LIMIT 5' \
, connection)

,order_id,product_id,quantity,unit_price,product_id,name,quantity_in_stock,unit_price
0,2,1,2,9.10,1,Foam Dinner Plate,70,1.21
1,6,1,4,8.65,1,Foam Dinner Plate,70,1.21
2,10,1,10,6.01,1,Foam Dinner Plate,70,1.21
3,5,2,3,9.89,2,"Pork - Bacon,back Peameal",49,4.65
4,6,2,4,3.28,2,"Pork - Bacon,back Peameal",49,4.65


In [243]:
# join with the same table lets take sql_hr database with employees table 
# find the repots_to name (which is present in the same table)
connection.database = 'sql_hr'
pd.read_sql_query(
'SELECT  * \
FROM employees \
LIMIT 5' \
, connection)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1
3,40448,Mindy,Crissil,Staff Scientist,94860,37270.0,1
4,56274,Keriann,Alloisi,VP Marketing,110150,37270.0,1


In [244]:
pd.read_sql_query(
'SELECT  e.employee_id, e.first_name, m.first_name AS manager \
FROM employees e \
JOIN employees m ON e.reports_to = m.employee_id' \
, connection)

,employee_id,first_name,manager
0,33391,D'arcy,Yovonnda
1,37851,Sayer,Yovonnda
2,40448,Mindy,Yovonnda
3,56274,Keriann,Yovonnda
4,63196,Alaster,Yovonnda
5,67009,North,Yovonnda
6,67370,Elladine,Yovonnda
7,68249,Nisse,Yovonnda
8,72540,Guthrey,Yovonnda
9,72913,Kass,Yovonnda


## Multiple Table Joins

In [250]:
# we will use sql_store - orders, customers and order_satuses to create a report which shows 
connection.database = 'sql_store'
pd.read_sql_query(
'SELECT  o.order_id, o.order_date, c.first_name, os.name AS status \
FROM orders o \
JOIN customers c ON o.customer_id = c.customer_id \
JOIN order_statuses os ON o.status = os.order_status_id' \
, connection)
# pd.read_sql_query(
# 'SELECT  * \
# FROM orders' \
# , connection)

,order_id,order_date,first_name,status
0,1,2019-01-30,Elka,Processed
1,3,2017-12-01,Thacher,Processed
2,4,2017-01-22,Ines,Processed
3,6,2018-11-18,Levy,Processed
4,8,2018-06-08,Clemmie,Processed
5,2,2018-08-02,Ilene,Shipped
6,5,2017-08-25,Clemmie,Shipped
7,7,2018-09-22,Ines,Shipped
8,9,2017-07-05,Levy,Shipped
9,10,2018-04-22,Elka,Shipped


## Compound Join Conditions
Compound join are required where you cannot identify the table record uniquely with a simple column. So if the values in a column are repeated we cannot perform a join on that column. SO we will need compound join conditions.


In [295]:
pd.read_sql_query(
'SELECT  * \
FROM order_item_notes' \
, connection)

,note_id,order_Id,product_id,note
0,1,1,2,first note
1,2,1,2,second note


In [296]:
print(pd.read_sql_query(
'SELECT  * \
FROM order_items \
LIMIT 100' \
, connection))
print(pd.read_sql_query(
'SELECT  * \
FROM order_item_notes \
LIMIT 100' \
, connection))

    order_id  product_id  quantity  unit_price
0          1           4         4        3.74
1          2           1         2        9.10
2          2           4         4        1.66
3          2           6         2        2.94
4          3           3        10        9.12
5          4           3         7        6.99
6          4          10         7        6.40
7          5           2         3        9.89
8          6           1         4        8.65
9          6           2         4        3.28
10         6           3         4        7.46
11         6           5         1        3.45
12         7           3         7        9.17
13         8           5         2        6.94
14         8           8         2        8.59
15         9           6         5        7.28
16        10           1        10        6.01
17        10           9         9        4.28
   note_id  order_Id  product_id         note
0        1         1           2   first note
1        2     

In [373]:
# Example in sql_stores.order_items. column order_id has duplicates, so does product_id. SO we will use a combination of both.
# Basically that table has a composite primary key
# lets JOIN order_items to order_item_notes
pd.read_sql_query(
'SELECT  * \
FROM order_items oi \
JOIN order_item_notes oin ON oi.order_id = oin.order_id AND oi.product_id = oin.product_id' \
, connection)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,1,4,4,3.74,1,1,4,first note


## implicit JOIN Syntax
Not a good practice to use.

In [300]:
pd.read_sql_query(
'SELECT  order_id, first_name, c.customer_id \
FROM orders o \
JOIN customers c ON o.customer_id = c.customer_id' \
, connection)
# IMplicit JOIN syntax
pd.read_sql_query(
'SELECT  order_id, first_name, c.customer_id \
FROM orders o, customers c \
WHERE o.customer_id = c.customer_id' \
, connection)

,order_id,first_name,customer_id
0,4,Ines,2
1,7,Ines,2
2,5,Clemmie,5
3,8,Clemmie,5
4,1,Elka,6
5,10,Elka,6
6,2,Ilene,7
7,3,Thacher,8
8,6,Levy,10
9,9,Levy,10


# OUTER JOIN


In [309]:
pd.read_sql_query(
'SELECT  c.customer_id, first_name, order_id \
FROM customers c \
JOIN orders o ON o.customer_id = c.customer_id \
ORDER BY c.customer_id' \
, connection)
# If you see the results only customer's with orders have been listed in the report. If a customer doesnt have a order 
# his or her record is not listed.

,customer_id,first_name,order_id
0,2,Ines,4
1,2,Ines,7
2,5,Clemmie,5
3,5,Clemmie,8
4,6,Elka,1
5,6,Elka,10
6,7,Ilene,2
7,8,Thacher,3
8,10,Levy,6
9,10,Levy,9


In [318]:
# Lets do a LEFT JOIN to solve the above problem
# LEFT JOIN gives all records from LEFT table is returned irrespective of the table.
pd.read_sql_query(
'SELECT  c.customer_id, first_name, order_id \
FROM customers c \
LEFT JOIN orders o ON c.customer_id = o.customer_id \
ORDER BY c.customer_id' \
, connection)
# We can use RIGHT JOIN if we want the other way around. Change and see.
# Basically LEFT JOIN == LEFT OUTER JOIN
# RIGHT JOIN == RIGHT OUTER JOIN

,customer_id,first_name,order_id
0,1,Babara,NaN
1,2,Ines,4.0
2,2,Ines,7.0
3,3,Freddi,NaN
4,4,Ambur,NaN
5,5,Clemmie,5.0
6,5,Clemmie,8.0
7,6,Elka,1.0
8,6,Elka,10.0
9,7,Ilene,2.0


In [325]:
# Outer join between multiple table
# Same as above added another inner join. This will return records for which there si a shipper id. 
pd.read_sql_query(
'SELECT  c.customer_id, first_name, order_id, o.shipper_id, sh.name \
FROM customers c \
LEFT JOIN orders o ON c.customer_id = o.customer_id \
JOIN Shippers sh ON o.shipper_id = sh.shipper_id \
ORDER BY c.customer_id' \
, connection)

,customer_id,first_name,order_id,shipper_id,name
0,2,Ines,7,4,"Mraz, Renner and Nolan"
1,5,Clemmie,5,3,Satterfield LLC
2,6,Elka,10,2,Schinner-Predovic
3,7,Ilene,2,4,"Mraz, Renner and Nolan"
4,10,Levy,9,1,Hettinger LLC


In [327]:
# To get all the records do a LEFT JOIN. Change to RIGHT JOIN to see only shippers table record.
pd.read_sql_query(
'SELECT  c.customer_id, first_name, order_id, o.shipper_id, sh.name AS shipper \
FROM customers c \
LEFT JOIN orders o ON c.customer_id = o.customer_id \
LEFT JOIN Shippers sh ON o.shipper_id = sh.shipper_id \
ORDER BY c.customer_id' \
, connection)

,customer_id,first_name,order_id,shipper_id,shipper
0,1,Babara,NaN,NaN,None
1,2,Ines,4.0,NaN,None
2,2,Ines,7.0,4.0,"Mraz, Renner and Nolan"
3,3,Freddi,NaN,NaN,None
4,4,Ambur,NaN,NaN,None
5,5,Clemmie,5.0,3.0,Satterfield LLC
6,5,Clemmie,8.0,NaN,None
7,6,Elka,1.0,NaN,None
8,6,Elka,10.0,2.0,Schinner-Predovic
9,7,Ilene,2.0,4.0,"Mraz, Renner and Nolan"


In [356]:
# This is a example to show that multiple joins happen like this say for 
# A JOIN B JOIN C => JOIN(JOIN(A,B), C)
# result of first join between two table acts as left table to the second join.
pd.read_sql_query(
'SELECT o.order_id, o.order_date, c.first_name, sh.name AS shipper, os.name AS status \
FROM orders o \
JOIN customers c ON o.customer_id = c.customer_id \
LEFT JOIN shippers sh ON o.shipper_id = sh.shipper_id \
JOIN order_statuses os ON o.status = os.order_status_id \
ORDER BY o.order_id' \
, connection)

# JOIN order_statuses os ON o.status = os.order_status_id \

,order_id,order_date,first_name,shipper,status
0,1,2019-01-30,Elka,None,Processed
1,2,2018-08-02,Ilene,"Mraz, Renner and Nolan",Shipped
2,3,2017-12-01,Thacher,None,Processed
3,4,2017-01-22,Ines,None,Processed
4,5,2017-08-25,Clemmie,Satterfield LLC,Shipped
5,6,2018-11-18,Levy,None,Processed
6,7,2018-09-22,Ines,"Mraz, Renner and Nolan",Shipped
7,8,2018-06-08,Clemmie,None,Processed
8,9,2017-07-05,Levy,Hettinger LLC,Shipped
9,10,2018-04-22,Elka,Schinner-Predovic,Shipped


In [363]:
# If you want all the values from both the tables use FULL OUTER JOIN
# MySQL doesn't have syntax keyword FULL OUTER JOIN. You have to use combination of LEFT and RIGHT JOIN to obtain full joins.
# pd.read_sql_query(
# 'SELECT * \
# FROM customers c \
# FULL OUTER JOIN orders o ON c.customer_id = o.customer_id \
# ORDER BY c.customer_id' \
# , connection)
# c.customer_id, first_name, order_id

## USING Clause

In [370]:
# If we have same column names in both the tables its more cleaner to use USING than ON.
# ON and USING have the same effect just that USING can be used only when we have same coulmn names in both the tables.
pd.read_sql_query(
'SELECT o.order_id, c.first_name, sh.name AS shipper \
FROM customers c \
JOIN orders o \
    USING (customer_id) \
LEFT JOIN shippers sh \
    USING (shipper_id) \
JOIN order_statuses os ON o.status = os.order_status_id \
ORDER BY o.order_id' \
, connection)

# Following is replaced with USING
# ON c.customer_id = o.customer_id \ 
# ON o.shipper_id = sh.shipper_id \

,order_id,first_name,shipper
0,1,Elka,None
1,2,Ilene,"Mraz, Renner and Nolan"
2,3,Thacher,None
3,4,Ines,None
4,5,Clemmie,Satterfield LLC
5,6,Levy,None
6,7,Ines,"Mraz, Renner and Nolan"
7,8,Clemmie,None
8,9,Levy,Hettinger LLC
9,10,Elka,Schinner-Predovic


In [372]:
# we can do the same USING with compound join conditions (multiple columns in ab table)
pd.read_sql_query(
'SELECT  * \
FROM order_items oi \
JOIN order_item_notes oin \
    USING (order_id, product_id)' \
, connection)
# ON oi.order_id = oin.order_id AND oi.product_id = oin.product_id

,order_id,product_id,quantity,unit_price,note_id,note
0,1,4,4,3.74,1,first note


In [375]:
# NATURAL JOINS 
# the sql will look at the two tables and join them on common columns. NO need to specify the columns.
# THis is not a prefered way as sometimes things go out of control
pd.read_sql_query(
'SELECT c.customer_id, o.order_id, c.first_name \
FROM customers c \
NATURAL JOIN orders o \
ORDER BY c.customer_id' \
, connection)

,customer_id,order_id,first_name
0,2,4,Ines
1,2,7,Ines
2,5,5,Clemmie
3,5,8,Clemmie
4,6,1,Elka
5,6,10,Elka
6,7,2,Ilene
7,8,3,Thacher
8,10,6,Levy
9,10,9,Levy


In [379]:
# CROSS JOIN
# Every record (n) in one table will be combined with every record(m) in the other table, so it will have n*m rows
pd.read_sql_query(
'SELECT c.customer_id, o.order_id, c.first_name \
FROM customers c \
CROSS JOIN orders o \
ORDER BY c.customer_id' \
, connection)
# Same as above code but with Explicit syntax for cross joins
# pd.read_sql_query(
# 'SELECT c.customer_id, o.order_id, c.first_name \
# FROM customers c, orders o \
# ORDER BY c.customer_id' \
# , connection)

,customer_id,order_id,first_name
0,1,1,Babara
1,1,4,Babara
2,1,5,Babara
3,1,2,Babara
4,1,10,Babara
...,...,...,...
95,10,2,Levy
96,10,4,Levy
97,10,5,Levy
98,10,8,Levy


## UNION 
1. Joins are for combining 2 tables with columns
2. Unions for combining with rows. We can combine multiple queries using UNION to get the final result. The queries can be from same table or 2 differernt tables.


In [393]:
# this is not a idea example , but just for showing the usecase of UNION
pd.read_sql_query(
'SELECT c.first_name, c.city, c.state \
FROM customers c \
WHERE state = "VA" \
UNION \
SELECT c.first_name, c.city, c.state \
FROM customers c \
WHERE state = "IL"' \
, connection)

,first_name,city,state
0,Babara,Waltham,VA
1,Ines,Hampton,VA
2,Elka,Chicago,IL


In [408]:
# Better example, lets rank the customers as 'Bronze', 'Silver', 'GOld' according to their points
pd.read_sql_query(
'SELECT customer_id, first_name, points, "Bronze" AS type \
FROM customers \
WHERE points < 2000 \
UNION \
SELECT customer_id, first_name, points, "Silver" AS type \
FROM customers \
WHERE points BETWEEN 2000 AND 3000 \
UNION \
SELECT customer_id, first_name, points, "Gold" AS type \
FROM customers \
WHERE points > 3000 \
ORDER BY first_name' \
, connection)

,customer_id,first_name,points,type
0,4,Ambur,457,Bronze
1,1,Babara,2273,Silver
2,5,Clemmie,3675,Gold
3,6,Elka,3073,Gold
4,3,Freddi,2967,Silver
5,7,Ilene,1672,Bronze
6,2,Ines,947,Bronze
7,10,Levy,796,Bronze
8,9,Romola,1486,Bronze
9,8,Thacher,205,Bronze


## INSERT UPDATE and DELETE.
1. Very IMP ..for INSERT UPDATE and DELETE we cannot use pd.read_sql_query() its meant for reading probably not sure.
2. Use connection.cursor() for quering , inserting and updating the sql database, Reading with sursor is little difficult. We will have to fetch all the data's and it gives us like a tuple output. Better use pd.read for queries.
3. 

In [578]:
cursor = connection.cursor()

In [458]:
# use DESCRIBE to get the details of a table
# fields - table columns
# type - datatype of each column
cursor.execute(
'DESCRIBE customers'
)
cursor.fetchall()

[('customer_id', b'int', 'NO', 'PRI', None, 'auto_increment'),
 ('first_name', b'varchar(50)', 'NO', '', None, ''),
 ('last_name', b'varchar(50)', 'NO', '', None, ''),
 ('birth_date', b'date', 'YES', '', None, ''),
 ('phone', b'varchar(50)', 'YES', '', None, ''),
 ('address', b'varchar(50)', 'NO', '', None, ''),
 ('city', b'varchar(50)', 'NO', '', None, ''),
 ('state', b'char(2)', 'NO', '', None, ''),
 ('points', b'int', 'NO', '', b'0', '')]

In [460]:
pd.read_sql_query(
'DESCRIBE customers'
, connection)

,Field,Type,Null,Key,Default,Extra
0,customer_id,b'int',NO,PRI,None,auto_increment
1,first_name,b'varchar(50)',NO,,None,
2,last_name,b'varchar(50)',NO,,None,
3,birth_date,b'date',YES,,None,
4,phone,b'varchar(50)',YES,,None,
5,address,b'varchar(50)',NO,,None,
6,city,b'varchar(50)',NO,,None,
7,state,b'char(2)',NO,,None,
8,points,b'int',NO,,b'0',


In [548]:
pd.read_sql_query(
'SELECT * \
FROM orders' \
, connection)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [493]:
# Inserting a single row
cursor.execute('INSERT INTO customers \
VALUES (DEFAULT, "John", "Smith", "1990-01-01", NULL, "sasa lane", "Naperville", "IL", 234)')
connection.commit()

In [498]:
# optionaly you can supply the column names for which the values are passed. 
# Here we can skip some rows which can take NULL and Keys. So its easy to just sent the ones that are required.
cursor.execute('INSERT INTO customers (first_name, last_name, address, city, state) \
VALUES ("Rama", "Rao", "sasa lane", "Naperville", "IL")')
connection.commit()

In [490]:
# Inserting a multiple row
cursor.execute('INSERT INTO customers \
VALUES (DEFAULT, "Ravi", "Kal", "1990-01-01", "234-122-1000", "lili lane", "Naperville", "IL", 345), \
(DEFAULT, "Timi", "Ral", "1990-01-01", "345-122-1212", "lolo lane", "kuku", "IL", 233)')
connection.commit()

In [551]:
# Inserting hierarchical rows
# Inserting into multiple tables with orders Table being the parent and order_items table being the child.
# sql comes with some builtin functions like LAST_INSERT_ID which provide you access to the last inserted ID in order table
# which is required if we want to do mulitple table insertion
cursor.execute(
'INSERT INTO orders (customer_id, order_date, status) \
VALUES (1, "2019-01-02", 1); \
INSERT INTO order_items \
VALUES (SELECT LAST_INSERT_ID(), 1, 1, 2.95 ), \
(SELECT LAST_INSERT_ID(), 2, 1, 3.95 )')
connection.commit()
# LAST_INSERT_ID() - gives us the ID of the last inserted record in orders table
# Here the above code adds a record in order table then grabs the order_id (using LAST_INSERT_ID) and adds 
# 2 products in order_items table.
# this code is not working here, but will work well in sql workbench.

## Creating a copy of the table
Below code will work with little errors, just wont want to mess with the database here. Uncomment and run if required.

In [587]:
# Copy and archive the orders table
# cursor.execute(
# 'CREATE TABLE orders_archived AS \
# SELECT * FROM orders')
# connection.commit()
# This creates a archive table which is duplicate of orders but the archive table order_id is not marked as a primary key 
# and AutoInc is not set. 
# When we use a a query (in this case) SELECT statement inside another sql statement is is called as SOFT QUERY.

In [589]:
# pd.read_sql_query(
# 'DESCRIBE orders' \
# , connection)

In [576]:
# pd.read_sql_query(
# 'DESCRIBE orders_archived' \
# , connection)

,Field,Type,Null,Key,Default,Extra
0,order_id,b'int',NO,,b'0',
1,customer_id,b'int',NO,,None,
2,order_date,b'date',NO,,None,
3,status,b'tinyint',NO,,b'1',
4,comments,b'varchar(2000)',YES,,None,
5,shipped_date,b'date',YES,,None,
6,shipper_id,b'smallint',YES,,None,


In [563]:
# deleteing all teh records from the table
# This may not work so manually truncate in sql workbench
# cursor.execute('TRUNCATE orders_archived')
# connection.commit()

In [586]:
# not let make a soft query "SELECT" inside the INSERT to see how we can fill teh archive table.
# cursor.execute(
# 'INSERT INTO orders_archived \
# SELECT * FROM orders \
# WHERE order_date < "2019-01-01"')
# connection.commit()
# this will work just dont want to mess with the database here

## Update 

In [585]:
pd.read_sql_query(
'SELECT * FROM customers \
WHERE customer_id=1', connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,IL,5000


In [584]:
# Update single row
cursor.execute(
'UPDATE customers \
SET points=5000, state="IL" \
WHERE customer_id=1')
connection.commit()

In [592]:
pd.read_sql_query(
'SELECT * FROM customers \
WHERE city="Naperville"', connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,22,John,Smith,1990-01-01,234-122-1212,sasa lane,Naperville,IL,244
1,23,Ravi,Kal,1990-01-01,234-122-1000,lili lane,Naperville,IL,355
2,25,John,Smith,1990-01-01,None,sasa lane,Naperville,IL,244
3,26,Rama,Rao,None,None,sasa lane,Naperville,IL,10
4,27,Rama,Rao,None,None,sasa lane,Naperville,IL,10


In [593]:
# update multiple table
cursor.execute(
'UPDATE customers \
SET points=points+10 \
WHERE city="Naperville"')
connection.commit()
# if no WHERE clause, then the update will run on all the records

In [9]:
pd.read_sql_query(
'SELECT * FROM orders \
WHERE order_id=1', connection)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,updated 1,None,None


In [8]:
# subqueries in UPDATE 
# Here we are using the SELECT query inside a UPDATE query in its WHERE clause.
# So it collects the order_id from order_item_notes table to update the comment in orders table
cursor = connection.cursor() 
cursor.execute(
'UPDATE orders \
SET comments="updated 1" \
WHERE order_id = (SELECT order_Id FROM order_item_notes WHERE product_id=4)')
connection.commit()
# In case if the SELECT statement is returning multiple rcords we will have to use IN rather than = like this
# WHERE order_id IN (SELECT order_Id FROM order_item_notes WHERE product_id=4)')

## DELETE

In [46]:
pd.read_sql_query(
'SELECT * FROM customers', connection)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,IL,5000
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [ ]:
# Deleting all records from the table which you dont want to do
# cursor.execute(
# 'DELETE FROM cutomers')
# connection.commit()

In [17]:
# change the customer_id before running with a valid customer id
cursor.execute(
'DELETE FROM customers \
WHERE customer_id=27')
connection.commit()

In [48]:
# WE can use soft query inside DELETE as well. 
# THis is not a valid example, ut just for the sake.
# cursor.execute(
# 'DELETE FROM customers \
# WHERE customer_id IN (SELECT customer_id FROM customers WHERE customer_id NOT IN (SELECT customer_id FROM orders))')
# connection.commit()

In [53]:
connection.close()

# Access a Database with Python - Iris Dataset

The Iris dataset is a popular dataset especially in the Machine Learning community, it is a set of features of 50  Iris flowers and their classification into 3 species.
It is often used to introduce classification Machine Learning algorithms.

First let's download the dataset in `SQLite` format from Kaggle:

<https://www.kaggle.com/uciml/iris/>

Download `database.sqlite` and save it in the `data/iris` folder.

<p><img   src="https://upload.wikimedia.org/wikipedia/commons/4/49/Iris_germanica_%28Purple_bearded_Iris%29%2C_Wakehurst_Place%2C_UK_-_Diliff.jpg" alt="Iris germanica (Purple bearded Iris), Wakehurst Place, UK - Diliff.jpg" height="145" width="114"></p>

<p><br> From <a href="https://commons.wikimedia.org/wiki/File:Iris_germanica_(Purple_bearded_Iris),_Wakehurst_Place,_UK_-_Diliff.jpg#/media/File:Iris_germanica_(Purple_bearded_Iris),_Wakehurst_Place,_UK_-_Diliff.jpg">Wikimedia</a>, by <a href="//commons.wikimedia.org/wiki/User:Diliff" title="User:Diliff">Diliff</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="http://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=33037509">Link</a></p>

First let's check that the sqlite database is available and display an error message if the file is not available (`assert` checks if the expression is `True`, otherwise throws `AssertionError` with the error message string provided):

In [196]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [197]:
import os
data_iris_folder_content = os.listdir("data/iris")

In [198]:
error_message = "Error: sqlite file not available, check instructions above to download it"
assert "database.sqlite" in data_iris_folder_content, error_message

## Access the Database with the sqlite3 Package

We can use the `sqlite3` package from the Python standard library to connect to the `sqlite` database:

In [199]:
import sqlite3
import pandas as pd

In [200]:
conn = sqlite3.connect('data/iris/database.sqlite')
conn

In [201]:
cursor = conn.cursor()
cursor

In [202]:
type(cursor)

sqlite3.Cursor

A `sqlite3.Cursor` object is our interface to the database, mostly throught the `execute` method that allows to run any `SQL` query on our database.

First of all we can get a list of all the tables saved into the database, this is done by reading the column `name` from the `sqlite_master` metadata table with:

    SELECT name FROM sqlite_master
    
The output of the `execute` method is an iterator that can be used in a `for` loop to print the value of each row.

In [203]:
for row in cursor.execute("SELECT name FROM sqlite_master"):
    print(row)

('Iris',)
('grade',)


a shortcut to directly execute the query and gather the results is the `fetchall` method:

In [204]:
cursor.execute("SELECT name FROM sqlite_master").fetchall()

[('Iris',), ('grade',)]

**Notice**: this way of finding the available tables in a database is specific to `sqlite`, other databases like `MySQL` or `PostgreSQL` have different syntax.

Then we can execute standard `SQL` query on the database, `SQL` is a language designed to interact with data stored in a relational database. It has a standard specification, therefore the commands below work on any database.

If you need to connect to another database, you would use another package instead of `sqlite3`, for example:

* [MySQL Connector](https://dev.mysql.com/doc/connector-python/en/) for MySQL
* [Psycopg](http://initd.org/psycopg/docs/install.html) for PostgreSQL
* [pymssql](http://pymssql.org/en/stable/) for Microsoft MS SQL

then you would connect to the database using specific host, port and authentication credentials but then you could execute the same exact `SQL` statements.

Let's take a look for example at the first 3 rows in the Iris table:

In [205]:
sample_data = cursor.execute("SELECT * FROM Iris LIMIT 20").fetchall()

In [206]:
print(type(sample_data))
sample_data

<class 'list'>


[(1, 5.1, 3.5, 1.4, 0.2, 'Iris-setosa'),
 (2, 4.9, 3, 1.4, 0.2, 'Iris-setosa'),
 (3, 4.7, 3.2, 1.3, 0.2, 'Iris-setosa'),
 (4, 4.6, 3.1, 1.5, 0.2, 'Iris-setosa'),
 (5, 5, 3.6, 1.4, 0.2, 'Iris-setosa'),
 (6, 5.4, 3.9, 1.7, 0.4, 'Iris-setosa'),
 (7, 4.6, 3.4, 1.4, 0.3, 'Iris-setosa'),
 (8, 5, 3.4, 1.5, 0.2, 'Iris-setosa'),
 (9, 4.4, 2.9, 1.4, 0.2, 'Iris-setosa'),
 (10, 4.9, 3.1, 1.5, 0.1, 'Iris-setosa'),
 (11, 5.4, 3.7, 1.5, 0.2, 'Iris-setosa'),
 (12, 4.8, 3.4, 1.6, 0.2, 'Iris-setosa'),
 (13, 4.8, 3, 1.4, 0.1, 'Iris-setosa'),
 (14, 4.3, 3, 1.1, 0.1, 'Iris-setosa'),
 (15, 5.8, 4, 1.2, 0.2, 'Iris-setosa'),
 (16, 5.7, 4.4, 1.5, 0.4, 'Iris-setosa'),
 (17, 5.4, 3.9, 1.3, 0.4, 'Iris-setosa'),
 (18, 5.1, 3.5, 1.4, 0.3, 'Iris-setosa'),
 (19, 5.7, 3.8, 1.7, 0.3, 'Iris-setosa'),
 (20, 5.1, 3.8, 1.5, 0.3, 'Iris-setosa')]

In [207]:
# cursor.description - This read-only attribute provides the column names of the last query.
# https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.description
[row[0] for row in cursor.description] 

['Id',
 'SepalLengthCm',
 'SepalWidthCm',
 'PetalLengthCm',
 'PetalWidthCm',
 'Species']

It is evident that the interface provided by `sqlite3` is low-level, for data exploration purposes we would like to directly import data into a more user friendly library like `pandas`.

## Import data from a database to `pandas`

In [208]:
import pandas as pd

In [209]:
iris_data = pd.read_sql_query("SELECT * FROM Iris", conn)

In [210]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [211]:
iris_data.dtypes

Id                 int64
SepalLengthCm    float64
SepalWidthCm     float64
PetalLengthCm    float64
PetalWidthCm     float64
Species           object
dtype: object

`pandas.read_sql_query` takes a `SQL` query and a connection object and imports the data into a `DataFrame`, also keeping the same data types of the database columns. `pandas` provides a lot of the same functionality of `SQL` with a more user-friendly interface.

However, `sqlite3` is extremely useful for downsizing data **before** importing them in `pandas`.

For example you might have 1 TB of data in a table stored in a database on a server machine. You are interested in working on a subset of the data based on some criterion, unfortunately it would be impossible to first load data into `pandas` and then filter them, therefore we should tell the database to perform the filtering and just load into `pandas` the downsized dataset.

In [212]:
iris_setosa_data = pd.read_sql_query("SELECT * FROM Iris WHERE Species == 'Iris-setosa'", conn)
iris_setosa_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [213]:
# iris_setosa_data
print(iris_setosa_data.shape)
print(iris_data.shape)


(50, 6)
(150, 6)


### Creating a TABLE :


In [214]:
# Creating another table in the database: 
# this will throw a error if it already exists.
# cursor.execute('CREATE TABLE grade(id int, name text, score int)')

In [215]:
import sqlite3
conn = sqlite3.connect('data/iris/database.sqlite')
cursor = conn.cursor()
cursor.execute('SELECT name FROM sqlite_master').fetchall()

[('Iris',), ('grade',)]

In [216]:
cursor.execute('DELETE FROM grade')
conn.commit()

In [217]:
# Add values to database One Row at a time
cursor.execute("INSERT INTO grade(id, name, score) VALUES(101, 'veena', 98)")
cursor.execute("INSERT INTO grade(id, name, score) VALUES(103, 'veena', 101)")


In [218]:
# Adding multiple rows at a time
data = [
    (104,'seema', 94),
    (106, 'naina', 87),
    (392, 'hassd', 34),
    (234, 'Naina', 455)
]
query_stmt = "INSERT INTO grade VALUES(?,?,?)"
cursor.executemany(query_stmt, data)

In [219]:
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [220]:
conn = sqlite3.connect('data/iris/database.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT * FROM grade").fetchall()

[(101, 'veena', 98),
 (103, 'veena', 101),
 (104, 'seema', 94),
 (106, 'naina', 87),
 (392, 'hassd', 34),
 (234, 'Naina', 455)]

In [221]:
pd.read_sql_query("SELECT * FROM grade", conn)
conn.close()

# MongoDB

1. MongoDB is a cross-platform document-oriented database program. Classified as a NoSQL database program, MongoDB uses JSON-like documents with optional schemas. The data is stored as Key-Value pairs.
2. Sql vs MongoDB database:
    * A SQL DB server consists of multiple DBs and each DB will have multiple tables. Each tables in turn will have multiple Rows.
    * A mongoDB server will also consists of multiple DBs and each DB will have multiple __Collections__ (Here Collections is used instrad of Tables). Each collection will have __Records/Documents__ (equivalent to rows). Each record or document is key value pay.

1. Download the latest "Mongo DB community server" version and install MongoDB. Installing as 'server' is recomended, so you will not have to run the mongo db server everytime to use it.
2. Open cmd and go to 'C:\Program Files\MongoDB\Server\4.2\bin' and hit mongo to confirm that we have installed it.If the command runs successfully, it means that the server is up and running, and we can proceed. You can also check to see that the server is runnign by going to 'Service' in start and look for MongoBD which says it is started.
<img src='images/image65.png' align='middle' style='width:500px; height:300px' />
3. Mongo Compass comes installed wiht MongoDB which is a gui for it.
4. Go the services section and then start the MongoDB service if not already started.
<img src='images/image64.png' align='middle' style='width:500px; height:100px' />
5. 

In [222]:
import pymongo # package which enables us to connect to mongodb
dbConn = pymongo.MongoClient('mongodb://localhost:27017') # connect to local running mongodb instance
dbConn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [223]:
print(dbConn.list_database_names())
db = dbConn['DemoDB'] # connect to DB named 'NamedDB' present in DB.
print(db)
print(db.list_collection_names())
collection = db['student'] # obtain the collection(table) name

['admin', 'config', 'crawlerDB', 'local']
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'DemoDB')
[]


In [224]:
# single insert
entry1 = {
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'yob': 1988
}
collection.insert_one(entry1) # insert a record, mydict is a dict with key-value pair

In [225]:
# multiple insert
entry2 = [
    {
    'name': 'Seema Mehar',
    'rollNo': 56789,
    'email': 'seema.meh@gmail.com',
    'major': 'Electronics',
    'yob': 1978
    },
    {
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'yob': 1985
    },
    {
    'name': 'Geet Handa',
    'rollNo': 5487,
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'yob': 1989
    }
    
]
collection.insert_many(entry2)

In [226]:
# retieving all the records from collection
records = collection.find({}) 
for record in records:
    print(record)

{'_id': ObjectId('5f4ea7c1aa5991e95d4a479a'), 'name': 'Veena Kalburgi', 'rollNo': 12345, 'email': 'veena.kal@gmail.com', 'major': 'Computer Science', 'yob': 1988}
{'_id': ObjectId('5f4ea7c1aa5991e95d4a479b'), 'name': 'Seema Mehar', 'rollNo': 56789, 'email': 'seema.meh@gmail.com', 'major': 'Electronics', 'yob': 1978}
{'_id': ObjectId('5f4ea7c1aa5991e95d4a479c'), 'name': 'Maan Khurana', 'rollNo': 26544, 'email': 'maan.geet@gmail.com', 'major': 'Computer Science', 'yob': 1985}
{'_id': ObjectId('5f4ea7c1aa5991e95d4a479d'), 'name': 'Geet Handa', 'rollNo': 5487, 'email': 'geet.maan@gmail.com', 'major': 'Stylish', 'yob': 1989}


In [227]:
# retrieveing some columns
students = collection.find({}, {'name'}).limit(5) # retrieveing two columns
# The second parameter in find() specifies which columns to choose
for student in students:
#     print(type(student)) # will give a dict
    print(student['name'])

Veena Kalburgi
Seema Mehar
Maan Khurana
Geet Handa


In [228]:
# query for data
my_query = {'yob':1988}
res = collection.find(my_query, {'name', 'yob'})

for r in res:
    pp.pprint(r)

{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479a'),
    'name': 'Veena Kalburgi',
    'yob': 1988}


In [229]:
# querey for data with condition
my_query = {'yob': {'$gt': 1980}} # yob is greater than 1980
res = collection.find(my_query).sort('yob', -1) # sort by 'yob' in descending order (-1)

for r in res:
    pp.pprint(r)

{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479d'),
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479a'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479c'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}


In [230]:
# delete records
del_stud = {'name': 'Seema Mehar'}
collection.delete_one(del_stud)
for rec in collection.find({}):
    pp.pprint(rec)

{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479a'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479c'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479d'),
    'email': 'geet.maan@gmail.com',
    'major': 'Stylish',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}


In [231]:
# for dropping a collection
print(db.list_collection_names())
# db.drop_collection('test')

['student']


In [232]:
print(dbConn.list_database_names())
print(db.list_collection_names())

['DemoDB', 'admin', 'config', 'crawlerDB', 'local']
['student']


In [233]:
update_criteria={'major':'Stylish'}
new_values= { "$set": { "major": "Fashion Designer" } }
collection.update_many(update_criteria, new_values)
for rec in collection.find({}):
    pp.pprint(rec)

{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479a'),
    'email': 'veena.kal@gmail.com',
    'major': 'Computer Science',
    'name': 'Veena Kalburgi',
    'rollNo': 12345,
    'yob': 1988}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479c'),
    'email': 'maan.geet@gmail.com',
    'major': 'Computer Science',
    'name': 'Maan Khurana',
    'rollNo': 26544,
    'yob': 1985}
{   '_id': ObjectId('5f4ea7c1aa5991e95d4a479d'),
    'email': 'geet.maan@gmail.com',
    'major': 'Fashion Designer',
    'name': 'Geet Handa',
    'rollNo': 5487,
    'yob': 1989}


In [234]:
# dbConn.close()